In [100]:
#importing the libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle as pkl
import dill

In [101]:
#reading the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [102]:
#dropping irrelevant columns 
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [103]:
# Encoding categorical data
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo = ohe.fit_transform(df["Geography"].values.reshape(-1,1)).toarray()
geo = pd.DataFrame(geo,columns = ohe.get_feature_names_out(["Geography"]))
ohe.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [104]:
df.drop(['Geography'],axis=1,inplace=True)
df = pd.concat([df,geo],axis=1)

#encoding gender
def gender_encode(df):
    return df["Gender"].apply(lambda x: 1 if x =="Male" else 0)

df["Gender"] = gender_encode(df)

In [105]:
X = df.drop(['EstimatedSalary'],axis=1)
y = df['EstimatedSalary']

#splitting the dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

#scaling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [106]:
#building the model
import tensorflow as ts
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense # type: ignore
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard # type: ignore
import datetime

In [107]:
model = Sequential([
    Dense(256,activation='relu',input_shape=(12,)),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(1)]
)

c:\Users\rashm\Downloads\NLP_KN\ANNProjectTF\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [108]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_25 (Dense)                │ (None, 256)            │         3,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,545 (174.00 KB)

 Trainable params: 44,545 (174.00 KB)

 Non-trainable params: 0 (0.00 B)

In [109]:
from tensorflow.keras.optimizers import Adam # type: ignore
adam  =Adam(learning_rate = 0.001)

In [110]:
model.compile(optimizer=adam,loss="mean_absolute_error",metrics=["mse"])

In [111]:
log_dir = "reglogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq = 1)

In [112]:
#setup early stopping
es = EarlyStopping(monitor="val_loss",patience = 50,restore_best_weights=True) 

In [113]:
history = model.fit(X_train,y_train,validation_data =(X_test,y_test),epochs=200,callbacks = [tensorflow_callback,es])

Epoch 1/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 96511.0547 - mse: 12613105664.0000 - val_loss: 52786.4375 - val_mse: 3910514176.0000
Epoch 2/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51116.4883 - mse: 3559093248.0000 - val_loss: 49871.1602 - val_mse: 3378463488.0000
Epoch 3/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49839.3984 - mse: 3372704768.0000 - val_loss: 49806.9648 - val_mse: 3362430464.0000
Epoch 4/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 50024.0156 - mse: 3357430016.0000 - val_loss: 50031.1719 - val_mse: 3401405440.0000
Epoch 5/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49440.7422 - mse: 3317825024.0000 - val_loss: 49857.2656 - val_mse: 3365984256.0000
Epoch 6/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49406.5352 - mse: 3317075968.0000 - val_loss: 49820.7812 - val_mse: 3362580480.0000
Epoch 7/200
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49309.5352 - mse: 3299689728.0000 - val_loss: 49885.0156 - val_mse: 

In [114]:
#load tensor board
%reload_ext tensorboard

%tensorboard --logdir reglogs/fit/20240927-145920

ERROR: Failed to launch TensorBoard (exited with 1).

In [118]:
test_loss,test_mae = model.evaluate(X_test,y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: 49964.9297 - mse: 3382844928.0000


In [119]:
model.save('annr.h5')

In [120]:
with open('oher.pkl','wb') as f:
    pkl.dump(ohe,f)
with open('scr.pkl','wb') as f:
    pkl.dump(sc,f)